In [ ]:
import scipp as sc
from scipp.plot import plot
import numpy as np
import dataconfig # run make_config.py to create this
from graphical_reduction import q1d, load_data, run_reduction  
import ipywidgets as w
import IPython.display as disp
import warnings
from convert_helpers import ConvertWidget, TransformationWidget

In [ ]:
class ReductionWidget(w.Box):
    def __init__(self):
        super().__init__()
        self.sample = w.Text(description='Sample', value='49338')
        self.sample_trans = w.Text(description='Sample trans', value='49339')
        self.background = w.Text(description='Background', value='49334')
        self.background_trans = w.Text(description='Backg... trans', value='49335')
        self.load_button = w.Button(description='Load')
        self.load_button.on_click(self._on_load_button_clicked)
        self._button = w.Button(description='Process')
        self._button.on_click(self._on_process_button_clicked)
        self.output = w.Output(width='100%', height='100%')
        self.children = [w.VBox([w.HBox([self.sample, self.load_button]), w.HBox([self.sample_trans, self._button]), self.background, self.background_trans, self.output])]
        
    def _on_process_button_clicked(self, b):
        self.output.clear_output()
        sample_run_number = self.sample.value
        sample_transmission_run_number = self.sample_trans.value
        background_run_number = self.background.value
        background_transmission_run_number = self.background_trans.value
        
        with self.output:
            run_reduction(sample_run_number, sample_transmission_run_number, background_transmission_run_number,
                          background_run_number, loaded_data, path, moderator_file, direct_beam_file,
                          l_collimation, r1, r2, dr)
        
        
    def _on_load_button_clicked(self, b):
        global loaded_data
        self.output.clear_output()
        run_list = [self.sample.value, self.sample_trans.value, self.background.value, self.background_trans.value]
        with self.output:
            load_data(run_list, loaded_data, path)
            

In [ ]:
class PlotWidget(w.Box):
    def __init__(self):
        super().__init__()
        options = [key for key, item in globals().items() if isinstance(item, (sc.DataArray, sc.Dataset))]
        self._data_selector = w.Combobox(placeholder='Data to plot', options=options)
        self._button = w.Button(description='Plot')
        self._button.on_click(self._on_button_clicked)
        self.output = w.Output(width='100%', height='100%')
        self.children = [w.VBox([w.HBox([self._data_selector, self._button]), self.output])]
        
    def _on_button_clicked(self, b):
        self.output.clear_output()
        data_dict = {key: globals()[key] for key in "".join(self._data_selector.value.split()).split(',')}
        ds = sc.Dataset(data_dict)
        with self.output:
            plot(ds)

In [ ]:
# Set reduction settings.
path = dataconfig.data_root
direct_beam_file = 'DirectBeam_20feb_full_v3.dat'
moderator_file = 'ModeratorStdDev_TS2_SANS_LETexptl_07Aug2015.txt'
l_collimation = sc.Variable(value=5.0, unit=sc.units.m)
r2 = sc.Variable(value=4.0824829046386295/1000, unit=sc.units.m) # sample aperture radius
r1 = sc.Variable(value=14.433756729740645/1000, unit=sc.units.m) # source aperture radius
dr = sc.Variable(value=8.0/1000, unit=sc.units.m) # virtual ring width on detector
wavelength_bins = sc.Variable(
        dims=['wavelength'],
        unit=sc.units.angstrom,
        values=np.geomspace(0.9, 13.5, num=110))
loaded_data = {}

In [ ]:
disp.display(ReductionWidget())

In [ ]:
disp.display(PlotWidget())